In [69]:
import pandas as pd
from clickhouse_driver import Client
from clickhouse_driver.errors import ServerException
from si_prefix import si_format

client=Client(host='localhost')


def query(query,columns=None,settings=[]):
    for str in query.split(';\n'):
        if str.strip():
            try:
                result, columnsFromQuery = client.execute(str, settings=settings, with_column_types=True)
            except ServerException as e:
                display(e.message)
                return

            if columns:
                df = pd.DataFrame(result, columns=columns)
            else:
                df = pd.DataFrame(result, columns=[tuple[0] for tuple in columnsFromQuery])

            display(pd.DataFrame([[si_format(client.last_query.progress.rows),si_format(client.last_query.progress.bytes)+'B', si_format(client.last_query.elapsed)+'s']], columns=['Processed Rows', 'Processed Bytes', 'Time']))
            display(df)


from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_cell_magic
def clickhouse(line, cell=None):
    if cell:
        query(cell)
    else:
        query(line)





# Create Tables

In [8]:
%%clickhouse
CREATE TABLE IF NOT EXISTS glove
(
    `word` String,
    `vector` Array(Float32)
)
ENGINE = MergeTree
ORDER BY word;

CREATE TABLE IF NOT EXISTS planes (  		
  normal Array(Float32),	
  offset Array(Float32)
) 
ENGINE = MergeTree 
ORDER BY ();

CREATE TABLE IF NOT EXISTS glove_lsh
(
  `word` String,
  `vector` Array(Float32),
  `bits` UInt128
)
ENGINE = MergeTree
ORDER BY (bits, word)
SETTINGS index_granularity = 128


,Processed Rows,Processed Bytes,Time
0,0,0,0.077766


""


,Processed Rows,Processed Bytes,Time
0,0,0,0.041242


""


,Processed Rows,Processed Bytes,Time
0,0,0,0.041558


""


# Populate glove table

In [14]:
%%clickhouse

TRUNCATE TABLE glove;

INSERT INTO glove
WITH
  splitByChar(' ', line) AS cols,
  trim(cols[1]) as word,
  arraySlice(cols,2) AS vector
SELECT word, vector FROM file('glove.840B.300d.zip :: glove.840B.300d.txt','LineAsString')


,Processed Rows,Processed Bytes,Time
0,0,0,0.212193


""


,Processed Rows,Processed Bytes,Time
0,2196017,5646236541,128.738847


""


# Populate Planes and Glove_LSH Table

In [58]:
%%clickhouse

CREATE TABLE IF NOT EXISTS planes (  		
  normal Array(Float32),	
  offset Array(Float32)
) 
ENGINE = MergeTree 
ORDER BY ();


CREATE TABLE IF NOT EXISTS glove_lsh
(
  `word` String,
  `vector` Array(Float32),
  `bits` UInt128
)
ENGINE = MergeTree
ORDER BY (bits, word)
SETTINGS index_granularity = 128;


TRUNCATE TABLE planes;
TRUNCATE TABLE glove_lsh;

INSERT INTO planes SELECT v1 - v2 AS normal, (v1 + v2) / 2 AS offset
FROM
(
	SELECT
    	  min(vector) AS v1,
    	  max(vector) AS v2
	FROM
	(
    	  SELECT vector
    	  FROM glove
    	  ORDER BY rand() ASC
    	  LIMIT 256
	)
	GROUP BY intDiv(rowNumberInAllBlocks(), 2)
);


INSERT INTO glove_lsh
WITH
   128 AS num_bits,
   (
       SELECT
           groupArray(normal) AS normals,
           groupArray(offset) AS offsets
       FROM
       (
           SELECT *
           FROM planes
           LIMIT num_bits
       )
   ) AS partition,
   partition.1 AS normals,
   partition.2 AS offsets
SELECT
   word,
   vector,
   arraySum((normal, offset, bit) -> bitShiftLeft(toUInt128(dotProduct(vector - offset, normal) > 0), bit), normals, offsets, range(num_bits)) AS bits
FROM glove
SETTINGS max_block_size = 1000


,Processed Rows,Processed Bytes,Time
0,0,0,0.065164


""


,Processed Rows,Processed Bytes,Time
0,0,0,0.08638


""


,Processed Rows,Processed Bytes,Time
0,0,0,0.053182


...

""


,Processed Rows,Processed Bytes,Time
0,2196017,2652788536,1.726412


""


,Processed Rows,Processed Bytes,Time
0,2196145,2690402599,156.878412


""


# Queries

## Naive Query

In [72]:
%%clickhouse

WITH
   'dog' AS search_term,
   (
       SELECT vector
       FROM glove
       WHERE word = search_term
       LIMIT 1
   ) AS target_vector
SELECT
   word,
   cosineDistance(vector, target_vector) AS score
FROM glove
WHERE lower(word) != lower(search_term)
ORDER BY score ASC
LIMIT 5

,Processed Rows,Processed Bytes,Time
0,2.2 M,2.7 GB,1.9 s


,word,score
0,dogs,0.116407
1,puppy,0.141479
2,pet,0.194255
3,cat,0.198315
4,puppies,0.248269


## Ordered by Hamming Distance

In [73]:
%%clickhouse

WITH 'dog' AS search_term,
   (
       SELECT vector
       FROM glove
       WHERE word = search_term
       LIMIT 1
   ) AS target_vector,
   128 AS num_bits,
   (
       SELECT
           groupArray(normal) AS normals,
           groupArray(offset) AS offsets
       FROM
       (
           SELECT *
           FROM planes
           LIMIT num_bits
       )
   ) AS partition,
   partition.1 AS normals,
   partition.2 AS offsets,
   (
       SELECT arraySum((normal, offset, bit) -> bitShiftLeft(toUInt128(dotProduct(target_vector - offset, normal) > 0), bit), normals, offsets, range(num_bits))
   ) AS target
SELECT word, bitHammingDistance(bits, target) AS distance
FROM glove_lsh WHERE word != search_term
ORDER BY distance ASC
LIMIT 20


,Processed Rows,Processed Bytes,Time
0,2.2 M,83.1 MB,173.0 ms


,word,distance
0,favourite.Seth,1
1,seeing,2
2,pleasure,2
3,reward,2
4,own,2
5,owning,2
6,road,2
7,pond,2
8,second-hand,2
9,toursWi-Fi,2


## Pre-Filtered by Hamming Distance

In [74]:
%%clickhouse

WITH
   'dog' AS search_term,
   (
       SELECT vector
       FROM glove
       WHERE word = search_term
       LIMIT 1
   ) AS target_vector,
   128 AS num_bits,
   (
       SELECT
           groupArray(normal) AS normals,
           groupArray(offset) AS offsets
       FROM
       (
           SELECT *
           FROM planes
           LIMIT num_bits
       )
   ) AS partition,
   partition.1 AS normals,
   partition.2 AS offsets,
   (
       SELECT arraySum((normal, offset, bit) -> bitShiftLeft(toUInt128(dotProduct(target_vector - offset, normal) > 0), bit), normals, offsets, range(num_bits))
   ) AS target
SELECT
   word,
   bitHammingDistance(bits, target) AS approx_distance,
   cosineDistance(vector, target_vector) AS score
FROM glove_lsh
PREWHERE approx_distance <= 5
WHERE word != search_term
ORDER BY score ASC
LIMIT 5

,Processed Rows,Processed Bytes,Time
0,2.2 M,81.1 MB,186.5 ms


,word,approx_distance,score
0,dogs,4,0.116407
1,puppy,3,0.141479
2,pet,3,0.194255
3,cat,2,0.198315
4,puppies,3,0.248269


## Simpler LSH

In [88]:
%%clickhouse

CREATE TABLE IF NOT EXISTS planes_simple (  		
  `projection` Array(Float32)
) 
ENGINE = MergeTree 
ORDER BY ();

TRUNCATE TABLE planes_simple;

INSERT INTO planes_simple SELECT projection / L2Norm(projection) AS projection
FROM
(
    SELECT arrayJoin(arraySplit((x, y) -> y, groupArray(e), arrayMap(x -> ((x % 300) = 0), range(128 * 300)))) AS projection
    FROM
    (
        SELECT CAST(randNormal(0, 1), 'Float32') AS e
        FROM numbers(128 * 300)
    )
);

CREATE TABLE IF NOT EXISTS glove_lsh_simple
(
  `word` String,
  `vector` Array(Float32),
  `bits` UInt128
)
ENGINE = MergeTree
ORDER BY (bits, word)
SETTINGS index_granularity = 128;

TRUNCATE TABLE glove_lsh_simple;

INSERT INTO glove_lsh_simple
WITH
    128 AS num_bits,
    (
        SELECT groupArray(projection) AS projections
        FROM
        (
            SELECT *
            FROM planes_simple
            LIMIT num_bits
        )
    ) AS projections
SELECT
    word,
    vector,
    arraySum((projection, bit) -> bitShiftLeft(toUInt128(dotProduct(vector, projection) > 0), bit), projections, range(num_bits)) AS bits
FROM glove
SETTINGS max_block_size = 1000;




,Processed Rows,Processed Bytes,Time
0,0.0,0.0 B,100.3 ms


""


,Processed Rows,Processed Bytes,Time
0,0.0,0.0 B,47.0 ms


""


,Processed Rows,Processed Bytes,Time
0,38.4 k,307.2 kB,61.6 ms


...

""


,Processed Rows,Processed Bytes,Time
0,0.0,0.0 B,57.8 ms


""


,Processed Rows,Processed Bytes,Time
0,2.2 M,2.7 GB,87.9 s


""


In [91]:
%%clickhouse

WITH
    'dog' AS search_term,
    (
        SELECT vector
        FROM glove
        WHERE word = search_term
        LIMIT 1
    ) AS target_vector,
    128 AS num_bits,
    (
        SELECT groupArray(projection) AS projections
        FROM
        (
            SELECT *
            FROM planes_simple
            LIMIT num_bits
        )
    ) AS projections,
    (
        SELECT arraySum((projection, bit) -> bitShiftLeft(toUInt128(dotProduct(target_vector, projection) > 0), bit), projections, range(num_bits))
    ) AS target
SELECT word, bitCount(bitXor(bits, target)) AS distance
FROM glove_lsh_simple
WHERE word != search_term
ORDER BY  ASC
LIMIT 5


,Processed Rows,Processed Bytes,Time
0,2.2 M,83.0 MB,183.9 ms


,word,distance
0,cat,13
1,puppy,14
2,pet,16
3,dogs,17
4,cats,22


In [93]:
%%clickhouse

CREATE TABLE IF NOT EXISTS glove_lsh16 (
    `word` String,
    `vector` Array(Float32),
    `bits` UInt128)
    ENGINE = MergeTree
    ORDER BY (bits, word)
    SETTINGS index_granularity = 128;

TRUNCATE TABLE glove_lsh16;

INSERT INTO glove_lsh16
WITH 16 AS num_bits,
     (SELECT 
        groupArray(normal) AS normals,
        groupArray(offset) AS offsets
        FROM (
            SELECT *
            FROM planes
            LIMIT num_bits
        )
    ) AS partition,
    partition.1 AS normals,
    partition.2 AS offsets
SELECT word, vector, arraySum((normal, offset, bit) -> bitShiftLeft(toUInt128(dotProduct(vector - offset, normal) > 0), bit), normals, offsets, range(num_bits)) AS bits
FROM glove
SETTINGS max_block_size = 1000




,Processed Rows,Processed Bytes,Time
0,0.0,0.0 B,102.7 ms


""


,Processed Rows,Processed Bytes,Time
0,0.0,0.0 B,52.6 ms


""


,Processed Rows,Processed Bytes,Time
0,2.2 M,2.7 GB,35.7 s


""


In [102]:
%%clickhouse

WITH 'dog' AS search_term,
    (SELECT vector FROM glove WHERE word = search_term LIMIT 1) AS target_vector,
    16 AS num_bits,
    (SELECT groupArray(normal) AS normals,groupArray(offset) AS offsets FROM
     (SELECT * FROM planes LIMIT num_bits)) AS partition,
    partition.1 AS normals,
    partition.2 AS offsets,
    (SELECT arraySum((normal, offset, bit) -> bitShiftLeft(toUInt128(dotProduct(target_vector - offset, normal) > 0), bit), normals, offsets, range(num_bits))) AS target
    
SELECT 
    word,
    bitHammingDistance(bits, target) AS approx_distance,
    cosineDistance(vector, target_vector) AS score
FROM glove_lsh16
PREWHERE approx_distance <= 0
WHERE word != search_term
ORDER BY score ASC
LIMIT 5


,Processed Rows,Processed Bytes,Time
0,2.2 M,145.6 MB,216.3 ms


,word,approx_distance,score
0,dogs,0,0.116407
1,puppy,0,0.141479
2,pet,0,0.194255
3,cat,0,0.198315
4,puppies,0,0.248269


In [103]:
%%clickhouse

WITH 'dog' AS search_term,
    (SELECT vector FROM glove WHERE word = search_term LIMIT 1) AS target_vector,
    16 AS num_bits,
    (SELECT groupArray(normal) AS normals,groupArray(offset) AS offsets FROM
     (SELECT * FROM planes LIMIT num_bits)) AS partition,
    partition.1 AS normals,
    partition.2 AS offsets,
    (SELECT arraySum((normal, offset, bit) -> bitShiftLeft(toUInt128(dotProduct(target_vector - offset, normal) > 0), bit), normals, offsets, range(num_bits))) AS target
    
SELECT 
    word,
    cosineDistance(vector, target_vector) AS score
FROM glove_lsh16
PREWHERE bits=target
WHERE word != search_term
ORDER BY score ASC
LIMIT 5


,Processed Rows,Processed Bytes,Time
0,105.9 k,111.4 MB,206.2 ms


,word,score
0,dogs,0.116407
1,puppy,0.141479
2,pet,0.194255
3,cat,0.198315
4,puppies,0.248269


In [99]:
%%clickhouse

CREATE TABLE IF NOT EXISTS bits (bits UInt32) ENGINE=MergeTree ORDER BY ();

TRUNCATE TABLE bits;

insert into bits select distinct(bits) from glove_lsh16;



,Processed Rows,Processed Bytes,Time
0,0.0,0.0 B,128.8 ms


""


,Processed Rows,Processed Bytes,Time
0,0.0,0.0 B,46.0 ms


""


,Processed Rows,Processed Bytes,Time
0,2.2 M,35.1 MB,119.8 ms


""
